# Функции в Pandas
Примеры из презентации

In [39]:
import pandas as pd

In [ ]:
df = pd.DataFrame({'user_id': [1, 2, 3], 'clicks': [163, 130, 97], 'orders': [2, 4, 0]})
df

In [ ]:
df = pd.DataFrame({'user_id': [1, 2, 3], 'clicks': [163, 130, 97], 'orders': [2, 4, 0], 'calculated': [False, False, True]})
df[['user_id', 'clicks', 'orders', 'calculated']]

In [ ]:
def watcher(param):
    """Мне только посмотреть"""
    return param == 0

In [ ]:
df['watcher'] = df['orders'].apply(watcher)
df

Применяем метод apply к одному столбцу. Сейчас в переменную функции передаются значения одного столбца

In [ ]:
def conversion(row):
    """Подсчет конверсии переходов в покупки"""
    return row['orders'] / row['clicks']

### Как потестировать функцию со значениями одной строки

In [ ]:
row1 = pd.DataFrame({'clicks': [163], 
                     'orders': [2], 
                     'user_id': [1]})
conversion(row1)

In [ ]:
row1

Применяем метод apply к датафрейму. В переменную функции передаются строки целиком

In [ ]:
df = pd.DataFrame({'user_id': [1, 2, 3], 'clicks': [163, 130, 97], 'orders': [2, 4, 0]})
df[['user_id', 'clicks', 'orders']]

In [ ]:
df['conversion'] = df.apply(conversion, axis=1)

In [ ]:
df[['user_id', 'clicks', 'orders', 'conversion']]

### Анализ рекламных кампаний

In [ ]:
import pandas as pd

In [ ]:
stats = pd.read_excel('ad_campaigns.xlsx', engine='openpyxl')
stats.head()

In [ ]:
stats.columns = ['group', 'phrase', 'effect', 'ad_id', 'title', 'text', 'link', 'n1', 'n2']
stats.head()

In [ ]:
stats = stats[['group', 'phrase', 'effect', 'ad_id', 'title', 'text', 'link']]
stats.head()

### Lambda-функции
Хотим посчитать распределение количества слов в столбце с фразами

In [ ]:
phrase = 'МРТ на Менделеевской от 2000'

In [ ]:
phrase.split(' ')

In [ ]:
len(phrase.split(' '))

In [ ]:
def word_count(phrase):
    return len(phrase.split(' '))

In [ ]:
word_count('МРТ на Менделеевской от 2000')

In [ ]:
word_count = lambda phrase: len(phrase.split(' '))

In [ ]:
stats['word_count'] = stats['phrase'].apply(lambda word: len(word.split(' ')))
stats.head()

In [ ]:
# вариант с передачей всей строчки функции
# тут надо обязательно указать параметр axis = 1

stats['word_count'] = stats.apply(lambda x: len(x['phrase'].split(' ')), axis=1)
stats.head()

### Упражнение
Напишите отдельную функцию word_count, которая считает количество слов в столбце phrase. Функцию можно применять как к столбцу phrase, так и к строке целиком.

### Произвольные функции
В URL кампаний есть названия. С этим надо что-то делать

In [ ]:
# обращение к индексу и значениям Series
res = stats.word_count.value_counts()
res

In [ ]:
res.index[0]

In [ ]:
res.values[0]

In [ ]:
# пример ссылки
url = stats.loc[0, 'link']
url

In [ ]:
from urllib import parse

In [ ]:
parse.urlsplit('https://ya.ru/news/sport?search=footbal#abc')

In [ ]:
parse.urlsplit('https://ya.ru/news/sport?search=footbal#abc').netloc

In [ ]:
parsed = parse.urlsplit(url)
parsed

In [ ]:
# можно конечно вручную
parsed.query.split('&')[2].split('=')[1]

In [ ]:
# как доставать значения

parsed.netloc

In [ ]:
params = parse.parse_qs(parsed.query)
params

In [ ]:
# вот и кампании

params['utm_campaign'][0]

In [ ]:
# зачем тут везде списки?

url_with_doubles = 'https://awesome-site.ru/?a=1&a=2&a=3'

parsed = parse.urlsplit(url_with_doubles)
parse.parse_qs(parsed.query)

In [ ]:
# оборачиваем все в функцию
# в качестве аргумента будет строка датафрейма

def campaign_name(row):
    """Получение названия кампании из ссылки внутри строки row"""

    parsed = parse.urlsplit(row['link'])
    params_dict = parse.parse_qs(parsed.query)

    return params_dict['utm_campaign'][0]

In [ ]:
# проверяем датафрейм
stats.head()

In [ ]:
stats['campaign'] = stats.apply(campaign_name, axis=1)
stats.head()

### Как передать в функцию несколько аргументов

In [ ]:
# как передать несколько аргументов

def power_up(row, n):
    """Возводит значение столбца effect в степень n"""
    return row['effect'] ** n

In [ ]:
stats['power_up'] = stats.apply(power_up, n=2, axis=1)
stats.head()

### Упражнение
В наших данных есть много объявлений с услугой МРТ (в столбце group есть слово 'мрт') круглосуточно (в тексте объявления text есть '24 часа'). Отфильтруйте строки, в которых НЕ упоминается МРТ, но прием идет круглосуточно. Сколько таких строк в датасете?

In [ ]:
def mrt(row):
    if ('мрт' not in row['group']) and ('24 часа' in row['text']):
        return True
    
    return False

In [ ]:
stats['mrt'] = stats.apply(mrt, axis=1)

In [ ]:
stats.head()

In [ ]:
stats.mrt.value_counts()

# Группировки

In [ ]:
df = pd.DataFrame({'order_id': [1, 2, 3, 4, 5], 'country': ['Россия', 'Китай', 'Китай', 'Россия', 'Россия'], 
                   'category': ['Электроника', 'Авто', 'Электроника', 'Авто', 'Авто'], 
                   'amount': [100, 80, 90, 140, 90]})
df

Создадим датафрейм df_russia, в котором оставим заказы из России. И аналогично df_china (заказы из Китая).

In [ ]:
df_russia = df[df.country == 'Россия']
df_russia

In [ ]:
df_china = df[df.country == 'Китай']
df_china

Посчитаем для df_russia и df_china:
- среднюю стоимость заказа
- разницу между максимальной и минимальной стоимостью заказа

In [ ]:
df_russia.amount.mean()

In [ ]:
df_china.amount.mean()

In [ ]:
df_russia.amount.max() - df_russia.amount.min()

In [ ]:
df_china.amount.max() - df_china.amount.min()

Объединим процесс разбиения на датафреймы

In [ ]:
def groupby_function(data):
    return data.amount.max() - data.amount.min()

In [ ]:
df.groupby('country').apply(groupby_function)

Вернемся к статистике рекламных кампаний

In [ ]:
# раньше использовали value_counts

stats['campaign'].value_counts().head()

In [ ]:
# более универсальный способ

stats.groupby('campaign').count().head()

In [ ]:
stats.head()

In [ ]:
stats.groupby('campaign').count()[['group', 'effect']].head()

### Как вернуть столбец из индекса - метод reset_index()

In [ ]:
stats.groupby('campaign').count().reset_index().head()

### К группировке можно применять разные функции такие образом:

In [ ]:
obj = stats.groupby('campaign')

In [ ]:
type(obj)

In [ ]:
obj.max().head()

In [ ]:
obj.mean().head()

### Несколько функций в группировках

In [ ]:
# задаем несколько функций сразу

stats.groupby('campaign').agg(['min', 'max'])['effect'].head()

In [ ]:
# разные метрики для разных столбцов

results = stats.groupby('campaign').agg({'effect': ['min', 'max'], 'power_up': 'mean'})
results.head()

### Как обращаться к вложенным столбцам

In [ ]:
results['effect']['max'].head()

In [ ]:
# или даже так
stats.groupby('campaign').agg({'effect': ['min', 'max'], 'power_up': 'mean'})['effect']['max'].head()

In [ ]:
# группировка по нескольким столбцам

stats.groupby(['group', 'campaign']).count().head()

In [ ]:
# максимальное число объявлений в одной группе

stats.groupby(['group', 'campaign']).count().sort_values('phrase', ascending=False).reset_index().head()

# Домашнее задание

## Задание 1

Напишите функцию, которая классифицирует фильмы из материалов занятия по следующим правилам:
- оценка 2 и меньше - низкий рейтинг
- оценка 4 и меньше - средний рейтинг
- оценка 4.5 и 5 - высокий рейтинг

Результат классификации запишите в столбец class

In [ ]:
rates = pd.read_csv('ratings.csv')

In [42]:
def rater(row):
    if row['rating'] <= 2: return 'Низкий'
    if row['rating'] <= 4: return 'Средний'
    if row['rating'] <= 5: return 'Высокий'

rates['class'] = rates.apply(rater, axis=1)
rates.head()

,userId,movieId,rating,timestamp,class
0,1,31,2.5,1260759144,Средний
1,1,1029,3.0,1260759179,Средний
2,1,1061,3.0,1260759182,Средний
3,1,1129,2.0,1260759185,Низкий
4,1,1172,4.0,1260759205,Средний


## Задание 2

Используем файл keywords.csv.

Необходимо написать гео-классификатор, который каждой строке сможет выставить географическую принадлежность определенному региону. Т. е. если поисковый запрос содержит название города региона, то в столбце ‘region’ пишется название этого региона. Если поисковый запрос не содержит названия города, то ставим ‘undefined’.

Правила распределения по регионам Центр, Северо-Запад и Дальний Восток:

```
geo_data = {
'Центр': ['москва', 'тула', 'ярославль'],
'Северо-Запад': ['петербург', 'псков', 'мурманск'],
'Дальний Восток': ['владивосток', 'сахалин', 'хабаровск']
}
```

Результат классификации запишите в отдельный столбец region.

In [43]:
keywords = pd.read_csv('keywords.csv')

In [58]:
geo_data = {
'Центр': ['москва', 'тула', 'ярославль'],
'Северо-Запад': ['петербург', 'псков', 'мурманск'],
'Дальний Восток': ['владивосток', 'сахалин', 'хабаровск']
}

plane_geo_data = {}
for key, values in geo_data.items():
    plane_geo_data.update({item: key for item in values})

city_set = set(plane_geo_data.keys())

In [59]:
def region_classifier(row):
    cities = set(row['keyword'].split(' ')).intersection(city_set)
    return ', '.join([plane_geo_data[city] for city in cities])

keywords['region'] = keywords.apply(region_classifier, axis=1)
keywords[keywords.region.str.contains(', ')].head()

,keyword,shows,region
48830,сапсан москва санкт петербург,10469,"Северо-Запад, Центр"
52463,москва санкт петербург,9756,"Северо-Запад, Центр"


## Задание 3 (бонусное)

Есть мнение, что “раньше снимали настоящее кино, не то что сейчас”. Ваша задача проверить это утверждение, используя файлы с рейтингами фильмов из прошлого домашнего занятия (файл movies.csv и ratings.csv из базы https://grouplens.org/datasets/movielens). Т. е. проверить верно ли, что с ростом года выпуска фильма его средний рейтинг становится ниже.

При этом мы не будем затрагивать субьективные факторы выставления этих рейтингов, а пройдемся по следующему алгоритму:

1. В переменную years запишите список из всех годов с 1950 по 2010.

1. Напишите функцию production_year, которая каждой строке из названия фильма выставляет год выпуска. Не все названия фильмов содержат год выпуска в одинаковом формате, поэтому используйте следующий алгоритм:

   1. для каждой строки пройдите по всем годам списка years

   1. если номер года присутствует в названии фильма, то функция возвращает этот год как год выпуска

   1. если ни один из номеров года списка years не встретился в названии фильма, то возвращается 1900 год

1. Запишите год выпуска фильма по алгоритму пункта 2 в новый столбец ‘year’

1. Посчитайте средний рейтинг всех фильмов для каждого значения столбца ‘year’ и отсортируйте результат по убыванию рейтинга

In [63]:
import re

In [65]:
def production(row):
    years = re.findall(r'^.*(\d{4}).*$', row['title'])
    if years: return years[0]
    else: return '1900'

In [66]:
movies = pd.read_csv('movies.csv')
movies['year'] = movies.apply(production, axis=1)
movies.head()

,movieId,title,genres,year
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,1995
1,2,Jumanji (1995),Adventure|Children|Fantasy,1995
2,3,Grumpier Old Men (1995),Comedy|Romance,1995
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance,1995
4,5,Father of the Bride Part II (1995),Comedy,1995


In [67]:
df = rates.merge(movies, on='movieId')
df.head()

,userId,movieId,rating,timestamp,class,title,genres,year
0,1,31,2.5,1260759144,Средний,Dangerous Minds (1995),Drama,1995
1,7,31,3.0,851868750,Средний,Dangerous Minds (1995),Drama,1995
2,31,31,4.0,1273541953,Средний,Dangerous Minds (1995),Drama,1995
3,32,31,4.0,834828440,Средний,Dangerous Minds (1995),Drama,1995
4,36,31,3.0,847057202,Средний,Dangerous Minds (1995),Drama,1995


In [78]:
df.groupby('year').agg({'rating': ['mean']}).sort_values(('rating', 'mean'), ascending=False).head(120)

,rating
,mean
year,
1921,4.416667
1902,4.333333
1928,4.261905
1917,4.250000
1918,4.250000
...,...
2013,3.356973
2000,3.355945
